# This notebook illustrates the Gaussian Process Regression

In [118]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from jax import numpy as jnp
from jax import jit, grad,vmap
from InfoGrad import InfoGrad

import pickle as pkl

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
# The underlying function
def f(x):
    return -1+0.01*(x-4.9)*(x+5)*(x+0.5)*(x-3.5)


# The differentiable kernel function with parameters c,l not filled.
def k(x1,x2,c,l):
    small_sig = 1e-10 # This is needed for numerical stability.
    return c * jnp.exp(-(jnp.linalg.norm(x1-x2+small_sig,axis = -1)**2) / l)

def f_sampler(X):
    Y = f(X)
    return Y+np.random.randn(*Y.shape)*0.1

def random_sample_locs(n_locs,dim_lims):
    '''
        dim_lims = [(u1,l1),(u2,l2),...,(u{space_dim},l{space_dim})]
    '''
    return (np.random.rand(n_locs, len(dim_lims)))*(dim_lims[:,-1]-dim_lims[:,0]) + dim_lims[:,0]
    

In [120]:
space_dim = 1
step_size = 0.2
n_test = 100
dim_lims = np.array([(-5,5)])
var_0  = 0.01
N_iteration = 50


for mode in ['greedy','InfoGrad','randomWalk']:

# mode = 'InfoGrad'
# mode = 'randomWalk'

# X = np.linspace(-5,5,10)
    X = np.array([-5,0,5]).reshape(-1,space_dim)

    noisy_Y =  f_sampler(X)

    kernel = C(1.0, (1e-3, 1e3)) * RBF(100, (1e-6, 1e6))

    model = GPR(kernel,alpha = var_0,n_restarts_optimizer=20)

    model.fit(X,noisy_Y)

    curr_x = np.array([0.1,-0.1]).reshape(-1,space_dim)

    gradCalc = InfoGrad(k)

    n_0 = len(X)

    x_hist = []

    mu_hist = []

    std_hist = []


    full_X = np.linspace(-5,5,100)

    data = {}

    for n in range(N_iteration):
        new_x = []

        if mode == 'InfoGrad':
            dx = gradCalc.dIdx(model,curr_x,X.reshape(-1,space_dim)).reshape(curr_x.shape)
            new_x = curr_x + dx/np.linalg.norm(dx,axis=-1).reshape(curr_x.shape) * step_size
            
            new_x = np.array(new_x)
            new_x[new_x>5] = 5
            new_x[new_x<-5] = -5

            # Collect new data
            new_y = f_sampler(new_x)

            # Model update is mandatory, as required by sklearn.GPR

            X = np.vstack([X,new_x])
            noisy_Y = np.vstack([noisy_Y,new_y])

            model.fit(X,noisy_Y)

        elif mode == 'greedy':

            for i in range(len(curr_x)):

                x_test = random_sample_locs(n_test,dim_lims)
                _,std = model.predict(x_test,return_std = True)

                x_dest = x_test[np.argmax(std)]

                x_next = curr_x[i] + step_size * (x_dest-curr_x[i])/np.linalg.norm(x_dest-curr_x[i])
                
                x_next[x_next>5] = 5
                x_next[x_next<-5] = -5

                y_next = f_sampler(x_next)

                new_x.append(x_next)

                # Model update is mandatory, as required by sklearn.GPR

                X = np.vstack([X,x_next])
                noisy_Y = np.vstack([noisy_Y,y_next])

                model.fit(X,noisy_Y)
        elif mode == 'randomWalk':
            new_x = random_sample_locs(1,np.hstack([curr_x+step_size,curr_x-step_size])).reshape(curr_x.shape)

             # Collect new data
            new_y = f_sampler(new_x)

            # Model update is mandatory, as required by sklearn.GPR

            X = np.vstack([X,new_x])
            noisy_Y = np.vstack([noisy_Y,new_y])

            model.fit(X,noisy_Y)
        else:
            print('Mode {} does not exist.'.format(mode))
            break

        # Update curr_x

        curr_x = np.array(new_x)

        x_hist.append(np.array(new_x))

        mu,std= model.predict(full_X.reshape(-1,1),return_std=True)
        mu = mu.flatten()

        mu_hist.append(mu)
        std_hist.append(std)

    data['mu_hist'] = mu_hist
    data['std_hist'] = std_hist
    data['X'] = X
    data['noisy_Y'] = noisy_Y
    data['n_0'] = n_0
    data['n_robot'] = len(curr_x)


    with open('{}.pkl'.format(mode),'wb') as file:
        pkl.dump(data,file)

(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)
(2, 1, 2) (2, 2)


/home/tianpeng/miniconda3/envs/simulation/lib/python3.9/site-packages/sklearn/gaussian_process/_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


(2, 1, 2) (2, 2)
